In [13]:
import tensorflow as tf
import numpy as np
#from sklearn.model_selection import KFold
from keras.preprocessing import image
from keras.models import load_model, Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input

NameError: name 'tf' is not defined

In [ ]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [ ]:
with tf.device('/cpu:0'):
    base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
CLASSES = 7
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,        
        fill_mode='nearest',
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/home/marlan/data/ISIC2018/train/',
        target_size=(150, 150),
        batch_size=32,
        color_mode="rgb",
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/marlan/data/ISIC2018/test/',
        target_size=(150, 150),
        batch_size=32,
        color_mode="rgb",
        class_mode='categorical')

In [ ]:

EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS = 64
MODEL_FILE = 'scc.model'
history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    verbose=1,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS)
  


In [ ]:
model.save(MODEL_FILE)

In [4]:
model=load_model('scc.model')

In [21]:
img=image.load_img('/media/7D63-84F2/data/ISIC2018/IMG_0983.jpg',target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

In [22]:
preds = model.predict(img)

In [23]:
preds

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.3170508e-29, 1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [17]:
tf.keras.applications.inception_v3.decode_predictions(preds, top=7)

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 7)

In [ ]:
#img = image.load_img('test/Dog/110.jpg', target_size=(HEIGHT, WIDTH))
#preds = predict(load_model(MODEL_FILE), img)